# Decorators and Object-Oriented Programming


## The `@property` decorator

In some languages, it is _strongly discouraged_ to make any data attributes public, as we do here with a `Duck` class:

In [1]:
class DuckV1:
    def __init__(self, name):
        self.name = name
        
d1 = DuckV1('Donald')
d1.name

'Donald'

Let's say we have some code that _uses_ the `Duck` class:

In [2]:
def talk_like_a(duck, message):
    print(f'{duck.name} the Duck says "{message}"')
    
talk_like_a(d1, 'Hey there, Daisy!')

Donald the Duck says "Hey there, Daisy!"


Now let's imagine that we have allowed users to create their own `Duck`s, with the ability to update the duck's name. Unfortunately, our target audience is using "inappropriate" name, and, and we want to ensure that the duck names are "appropriate," so we replace the public attribute with a "getter" and a "setter":

In [3]:
class DuckV2:
    def __init__(self, name):
        self._name = name
        
    def get_name(self):
        return self._name
    
    def set_name(self, value):
        if value.lower() in ('porky', 'ed', 'bugs'):
            raise ValueError('Inappropriate name!')
        self._name = value
    
        
d2 = DuckV2('Donald')
d2.get_name()

'Donald'

In [4]:
d2.set_name('Ed')

ValueError: Inappropriate name!

The problem is that now, we've broken our `talk_like_a` function:

In [5]:
talk_like_a(d2, "Hey there, Daisy!")

AttributeError: 'DuckV2' object has no attribute 'name'

(and that's why some languages say to _start_ with getters and setters)

Python, however, has a solution for this: the `property` function creates a special "descriptor" object that allows us to use getters and setters that are accessed _just like attributes_:

In [6]:
class DuckV3:
    def __init__(self, name):
        self._name = name
        
    def get_name(self):
        return self._name
    
    def set_name(self, value):
        if value.lower() in ('porky', 'ed', 'bugs'):
            raise ValueError('Inappropriate name!')
        self._name = value
        
    name = property(get_name, set_name)
    
        
d3 = DuckV3('Donald')
d3.name

'Donald'

In [7]:
d3.name = 'Ed'

ValueError: Inappropriate name!

... and it works with our function now:

In [8]:
talk_like_a(d3, "Hey there, Daisy!")

Donald the Duck says "Hey there, Daisy!"


### The decorator syntax

The code above is *ok*, but could be improved:

- We don't need access to the old `get_name` and `set_name` functions any more since we can use them via the `property`
- The fact that the getter and setter are later wrapped up in a property is kind of obscured by being lower in the class definition.

To rectify this, we can use the decorator syntax:

If we type:

```python
@foo
def my_function():
    ...
```

then Python "re-writes" this as:

```python
def my_function():
    ...
my_function = foo(my_function)
```

### The @property decorator

The `property` function can be used as a decorator, as well, with the following syntax:

In [9]:
class DuckV4(object):
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        '''getter for name attribute'''
        return self._name
    
    @name.setter
    def name(self, val):
        '''setter for name attribute'''
        if val.lower() in ('porky', 'ed', 'bugs'):
            raise ValueError('Inappropriate name!')
        self._name = val
    

In [10]:
d4 = DuckV4('Donald')
d4.name

'Donald'

In [11]:
d4.name = 'Daffy'
d4.name

'Daffy'

In [12]:
d4.name = 'Bugs'

ValueError: Inappropriate name!

In [13]:
talk_like_a(d4, 'Hey there, Daisy!')

Daffy the Duck says "Hey there, Daisy!"


Practically speaking, we can also use `@property` to create computed attributes:

In [14]:
class Rect(object):
    
    def __init__(self, w, h):
        self.w, self.h = w, h
        
    @property
    def area(self):
        return self.w * self.h
        
r = Rect(3,4)

In [15]:
r.area

12

In [16]:
r.w = 4
r.area

16

In [17]:
r.area = 5

AttributeError: can't set attribute

We can even enhance the above by *caching* our computed attribute

In [18]:
class Rect(object):
    
    def __init__(self, w, h):
        self._w, self._h = w, h
        self._area = None
        
    @property
    def w(self):
        return self._w
    @w.setter
    def w(self, value):
        self._w = value
        self._area = None
        
    @property
    def h(self):
        return self._h
    @h.setter
    def h(self, value):
        self._h = value
        self._area = None
        
    @property
    def area(self):
        print('get area')
        if self._area is None:
            self._area = self._calc_area()
        return self._area
        
    def _calc_area(self):
        print('calc area')
        return self._w * self._h
    
r = Rect(3,4)


In [19]:
r.area

get area
calc area


12

In [20]:
r.__dict__

{'_w': 3, '_h': 4, '_area': 12}

In [21]:
r.area

get area


12

In [22]:
r.w = 4

In [23]:
r.__dict__

{'_w': 4, '_h': 4, '_area': None}

In [24]:
r.area

get area
calc area


16

In [25]:
r.area = 5

AttributeError: can't set attribute

## Class attributes vs. Instance attributes
* attributes set outside `__init__` belong to the *class* (as opposed to the *instance*) and are shared by all instances of the class
    * these attributes can be accessed via __`ClassName.var`__ and __`classInstance.var`__
* attributes created inside `__init__` (and all other method functions) and prefaced with __`self.`__ belong to the object *instance* and cannot be accessed via __`ClassName.`__

In [26]:
class Person(object):
    species = 'Human'
    
    def __init__(self, name):
        self.name = name
        print(
            "{}'s species is {}".format(
            self.name, self.species))

In [27]:
p1 = Person('Godzilla')

Godzilla's species is Human


In [28]:
Person.species, p1.species, p1.name

('Human', 'Human', 'Godzilla')

In [29]:
Person.name

AttributeError: type object 'Person' has no attribute 'name'

http://www.pythontutor.com/

In [30]:
p2 = Person('Mothra')
p2.name, p2.species

Mothra's species is Human


('Mothra', 'Human')

In [31]:
Person.species = 'animal'

In [32]:
p1.species, p2.species, Person.species

('animal', 'animal', 'animal')

In [33]:
p1.species = 'monster'

In [34]:
Person.species, p1.species, p2.species

('animal', 'monster', 'animal')

# Static and Class Methods
* static methods are methods that don't operate on an instance of the object and therefore are shared by all instances of the object
* class methods are methods that operate on the class itself, rather than instance of the class

In [35]:
class Duck(object):
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        '''getter for name attribute'''
        return self._name

    @name.setter
    def name(self, val):
        '''setter for name attribute'''
        self._name = val
    
    @staticmethod
    def myprint(thing):
        '''note that self is NOT the first param'''
        print('-' * len(thing))
        print(thing)
        print('-' * len(thing))

In [36]:
d = Duck('Donald')
d.myprint('Some thing')

----------
Some thing
----------


In [37]:
Duck.myprint('Some thing via the class')

------------------------
Some thing via the class
------------------------


In [38]:
class Duck(object):
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        '''getter for name attribute'''
        return self._name

    @name.setter
    def name(self, val):
        '''setter for name attribute'''
        self._name = val
    
    @staticmethod
    def myprint(thing):
        '''note that self is NOT the first param'''
        print('-' * len(thing))
        print(thing)
        print('-' * len(thing))
        
    @classmethod
    def myprint_cls(cls, thing):
        '''note that self is NOT the first param'''
        msg = f'{cls.__name__}: {thing}'
        print('-' * len(msg))
        print(msg)
        print('-' * len(msg))
        
    #myprint = staticmethod(myprint)

In [39]:
d = Duck('Donald')
d.myprint_cls('Some thing')

----------------
Duck: Some thing
----------------


In [41]:
class Mallard(Duck): pass

In [42]:
d = Mallard('Daffy')
d.myprint_cls('Some thing')

-------------------
Mallard: Some thing
-------------------


### class/static methods as "virtual constructors"

Suppose we want to let users create Ducks with a randomly-selected name. 

We *could* do this with a @staticmethod as follows:

In [43]:
import random

class Duck(object):
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        '''getter for name attribute'''
        return self._name

    @name.setter
    def name(self, val):
        '''setter for name attribute'''
        self._name = val
        
    @staticmethod
    def with_random_name(*options):
        name = random.choice(options)
        result = Duck(name)
        return result

In [50]:
d = Duck.with_random_name('Daffy', 'Donald', 'Scrooge')
d.name

'Daffy'

This breaks down with our Mallard class, though...

In [52]:
class Mallard(Duck): pass

In [53]:
d = Mallard.with_random_name('Daffy', 'Donald', 'Scrooge')
print(d.name, 'is a', type(d))

Daffy is a <class '__main__.Duck'>


We can fix this by making our method a @classmethod:

In [61]:
import random

class Duck(object):
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        '''getter for name attribute'''
        return self._name

    @name.setter
    def name(self, val):
        '''setter for name attribute'''
        self._name = val
        
    @classmethod
    def with_random_name(cls, *options):
        name = random.choice(options)
        result = cls(name)
        return result

In [62]:
d = Duck.with_random_name('Daffy', 'Donald', 'Scrooge')
d.name

'Daffy'

In [63]:
class Mallard(Duck): pass

In [64]:
d = Mallard.with_random_name('Daffy', 'Donald', 'Scrooge')
print(d.name, 'is a', type(d))

Daffy is a <class '__main__.Mallard'>


### class methods for accessing  class data

Although we can always "hard-code" the class name when accessing class data attributes directly, it's less fragile to use a `@classmethod`:

In [65]:
import random

class Duck(object):
    # Key is the name of the duck, value is the Duck
    _ducks = {}
    
    def __init__(self, name):
        self._name = name
        self._ducks[name] = self
        
    def __repr__(self):
        return f'<{type(self).__name__}: {self._name}>'
        
    @property
    def name(self):
        '''getter for name attribute'''
        return self._name

    @name.setter
    def name(self, val):
        '''setter for name attribute'''
        self._name = val
        
    @classmethod
    def by_name(cls, name):
        return cls._ducks[name]

In [70]:
Duck('Daffy')
Duck('Donald')
Duck('Dewey');

In [67]:
Duck.by_name('Donald')

<Duck: Donald>

In [68]:
Duck.by_name('Daffy')

<Duck: Daffy>

Now if we have a `Mallard` class, we can choose to either maintain all the Mallards in the `Duck` registry or create their own registry:

In [71]:
class SharedRegistryMallard(Duck): pass

class PrivateRegistryMallard(Duck):
    _ducks = {}  # Shadows the Duck._ducks class attribute

In [72]:
SharedRegistryMallard('Gus')
SharedRegistryMallard('Kishore')
PrivateRegistryMallard('Horatio')
PrivateRegistryMallard('Max');

In [73]:
Duck._ducks

{'Daffy': <Duck: Daffy>,
 'Donald': <Duck: Donald>,
 'Dewey': <Duck: Dewey>,
 'Gus': <SharedRegistryMallard: Gus>,
 'Kishore': <SharedRegistryMallard: Kishore>}

In [74]:
PrivateRegistryMallard._ducks

{'Horatio': <PrivateRegistryMallard: Horatio>,
 'Max': <PrivateRegistryMallard: Max>}

## Summary of instance, static, and class methods

In [91]:
class Foo(object):
    def __init__(self, name):
        self._name = name
    
    def __repr__(self):
        return f'<{self._name}>'
    
    def method(*args):
        print(f'method{args}')
        
    @staticmethod
    def static(*args):
        print(f'static{args}')
        
    @classmethod
    def clsmethod(*args):
        print(f'classmethod({args})')

In [92]:
bar = Foo('bar')

In [93]:
Foo.method(1, 2)

method(1, 2)


In [94]:
bar.method(1, 2)

method(<bar>, 1, 2)


In [95]:
Foo.static(1, 2)

static(1, 2)


In [96]:
bar.static(1, 2)

static(1, 2)


In [97]:
Foo.clsmethod(1, 2)

classmethod((<class '__main__.Foo'>, 1, 2))


In [98]:
bar.clsmethod(1, 2)

classmethod((<class '__main__.Foo'>, 1, 2))


# Lab

Open the [OOP Decorators Lab][oop-decorators-lab]

[oop-decorators-lab]: ./oop-decorators-lab.ipynb